In [22]:
from tqdm import tqdm_notebook
from google.cloud import bigquery
import polars as pl
import numpy as np
import pandas as pd
import random
import os
import sys
import gc

fast_experiment = True
type_labels = {'clicks':0, 'carts':1, 'orders':2}

In [40]:
%%time
# loading training data
client = bigquery.Client()
# define sql
sql = 'select session, aid, type, ts from `hsbc-1044360-ihubasp-sandbox.dna_openrice_scrapy.otto_train` order by session, datetime asc'
# Perform a query.
if fast_experiment:
    sql += ' limit 100000'
query_job = client.query((sql))  # API request
rows = query_job.result()  # Waits for query to finish

train_df = pl.from_arrow(rows.to_arrow())

CPU times: user 180 ms, sys: 51 ms, total: 231 ms
Wall time: 4.12 s


In [41]:
%%time
# convert ts and type
train_df = train_df.with_columns([
    (pl.col('ts') / 1000).cast(pl.Int32).alias('ts'),
    pl.col('type').apply(lambda t : type_labels[t]).cast(pl.Int8).alias('type')
])

train_df.head(3)

CPU times: user 326 ms, sys: 9.27 ms, total: 336 ms
Wall time: 331 ms


session,aid,type,ts
i64,i64,i8,i32
0,1517085,0,1659304800
0,1563459,0,1659304904
0,1309446,0,1659367439


### Generate user session features
- session length (numer of interaction per session) - can indicate whether this is a heavy customer or not
- number of clicks/carts/orders and ratio for each session - can indicate whether this customer browse a lot buy little or browse a lot and buy a lot
- transaction hours - can indicate what kind of this user is(AM or PM user), may help model to cluster customers

In [24]:
%%time
# user related features - session length
df_session_length = (
     train_df.select(pl.col(['session', 'aid'])).groupby('session').count()
)
df_session_length.head(3)

CPU times: user 8.86 ms, sys: 5.74 ms, total: 14.6 ms
Wall time: 2.92 ms


session,count
i64,u32
0,276
128,234
256,38


In [58]:
%%time
# user related features - number of clicks/carts/orders and its ratio for each session
df_session_type_count = (
    train_df.groupby(
        ['session', 'type']
    )
    .count()
    .pivot(
        values = 'count', index = 'session', columns='type', aggregate_fn='sum'
    )
    .with_column(
        pl.col("*").fill_null(pl.lit(0))
    )
    .rename(
        {
            '0':'clicks_per_session',
            '1':'carts_per_session',
            '2':'orders_per_session'
        }
    )
    .with_columns([
        pl.col("clicks_per_session").rank().alias('rank_clicks_per_session'),
        pl.col("carts_per_session").rank().alias('rank_carts_per_session'),
        pl.col("orders_per_session").rank().alias('rank_orders_per_session'),
        (pl.col('carts_per_session') / pl.col('clicks_per_session')).alias('cart_click_ratio'),
        (pl.col('orders_per_session') / pl.col('clicks_per_session')).alias('order_click_ratio'),
        (pl.col('orders_per_session') / pl.col('carts_per_session')).alias('order_cart_ratio')
    ])
)
df_session_type_count.head(3)

CPU times: user 47.5 ms, sys: 9.51 ms, total: 57 ms
Wall time: 8.53 ms


session,orders_per_session,clicks_per_session,carts_per_session,rank_clicks_per_session,rank_carts_per_session,rank_orders_per_session,cart_click_ratio,order_click_ratio,order_cart_ratio
i64,u32,u32,u32,f32,f32,f32,f64,f64,f64
561,5,213,17,1137.0,1123.5,1129.0,0.079812,0.023474,0.294118
19,7,97,8,934.0,966.0,1169.0,0.082474,0.072165,0.875
235,8,180,30,1103.0,1204.0,1186.5,0.166667,0.044444,0.266667


In [57]:
%%time
# user related features - user interaction hour - mean, min, max, std, rank across dataset
df_transaction_hour = (
    train_df.select(
        pl.col(['session', 'ts'])
    )
    .with_column(
        pl.from_epoch("ts", unit="s").alias('ts'),
    )
    .with_column(
        pl.col('ts').dt.hour().alias('hod') # hour of day
    )
    .select(
        pl.col(['session', 'hod'])
    )
    .groupby(
        'session'
    )
    .agg([
        pl.col('hod').mean().alias('mean_txn_hod'),
        pl.col('hod').median().alias('median_txn_hod'),
        pl.col('hod').min().alias('min_txn_hod'),
        pl.col('hod').max().alias('max_txn_hod'),
        pl.col('hod').std().alias('std_txn_hod')
    ])
    .with_columns([
        pl.col("mean_txn_hod").rank().alias('rank_mean_txn_hod'),
        pl.col("median_txn_hod").rank().alias('rank_median_txn_hod'),
        pl.col("min_txn_hod").rank().alias('rank_min_txn_hod'),
        pl.col("max_txn_hod").rank().alias('rank_max_txn_hod')
    ])
)

df_transaction_hour.head(3)

CPU times: user 26.7 ms, sys: 5 ms, total: 31.7 ms
Wall time: 6.94 ms


session,mean_txn_hod,median_txn_hod,min_txn_hod,max_txn_hod,std_txn_hod,rank_mean_txn_hod,rank_median_txn_hod,rank_min_txn_hod,rank_max_txn_hod
i64,f64,f64,u32,u32,f64,f32,f32,f32,f32
1024,11.532609,11.0,9,22,3.201823,69.0,108.0,729.0,468.0
1008,21.28125,22.0,10,22,2.17366,1015.0,1050.0,791.5,468.0
880,19.20339,22.0,12,23,4.151359,839.0,1050.0,868.0,1096.5
